<a href="https://colab.research.google.com/github/monoramasn/EDACC_WHISPER_MMS_SEAMLESS/blob/main/edacc_whisper_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/huggingface/peft.git@main
!pip install -U accelerate
!pip install evaluate
!pip install librosa
!pip install jiwer
!pip install --upgrade transformers bitsandbytes datasets torch torchvision torchaudio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Succes

In [2]:
import os
import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
import argparse
import evaluate
import re
from scipy import signal
import numpy as np
import pandas as pd
from dataclasses import dataclass
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from typing import Any, Dict, List, Union
from sklearn.model_selection import train_test_split
from datasets import load_dataset, DatasetDict, Audio, load_from_disk, concatenate_datasets, load_metric
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperTokenizer, WhisperFeatureExtractor
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [3]:
# Load datasets
edacc_dev = load_dataset("edinburghcstr/edacc", split="validation")
edacc_test = load_dataset("edinburghcstr/edacc", split="test")

# Create directories for each language in the test, train, and validation sets
base_dir = "organized_data"
os.makedirs(base_dir, exist_ok=True)

languages = edacc_test.unique("l1")
for split in ['test', 'train', 'validation']:
    for lang in languages:
        os.makedirs(os.path.join(base_dir, split, lang), exist_ok=True)

# Function to save audio data and corresponding text
def save_audio(example, folder):
    lang_folder = os.path.join(folder, example["l1"])
    os.makedirs(lang_folder, exist_ok=True)
    audio_filename = example['audio']['path'].split('/')[-1]
    audio_path = os.path.join(lang_folder, audio_filename)
    # Save audio file
    torchaudio.save(audio_path, torch.tensor(example['audio']['array']).unsqueeze(0), example['audio']['sampling_rate'])
    # Save corresponding text file
    text_path = os.path.join(lang_folder, f"{audio_filename}.txt")
    with open(text_path, "w") as f:
        f.write(example["text"])
    print(f"Saved audio: {audio_path}")
    print(f"Saved text: {text_path}")

# Save test audio and text to respective language directories
print("Saving test data...")
for example in edacc_test:
    save_audio(example, os.path.join(base_dir, 'test'))

# Split the validation dataset into train and validation sets
X = range(len(edacc_dev))
X_train, X_val = train_test_split(X, test_size=0.2, random_state=42)

train_dataset = edacc_dev.select(X_train)
val_dataset = edacc_dev.select(X_val)

# Save train audio and text to respective language directories
print("Saving train data...")
for example in train_dataset:
    save_audio(example, os.path.join(base_dir, 'train'))

# Save validation audio and text to respective language directories
print("Saving validation data...")
for example in val_dataset:
    save_audio(example, os.path.join(base_dir, 'validation'))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating validation split:   0%|          | 0/9848 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9289 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
Saved text: organized_data/train/Bulgarian/EDACC-C22-203.wav.txt
Saved audio: organized_data/train/Italian/EDACC-C01-146.wav
Saved text: organized_data/train/Italian/EDACC-C01-146.wav.txt
Saved audio: organized_data/train/Scottish English/EDACC-C46_P1-29.wav
Saved text: organized_data/train/Scottish English/EDACC-C46_P1-29.wav.txt
Saved audio: organized_data/train/Arabic/EDACC-C40_P3-40.wav
Saved text: organized_data/train/Arabic/EDACC-C40_P3-40.wav.txt
Saved audio: organized_data/train/Romanian/EDACC-C30-616.wav
Saved text: organized_data/train/Romanian/EDACC-C30-616.wav.txt
Saved audio: organized_data/train/Arabic/EDACC-C40_P3-110.wav
Saved text: organized_data/train/Arabic/EDACC-C40_P3-110.wav.txt
Saved audio: organized_data/train/Irish English/EDACC-C50-255.wav
Saved text: organized_data/train/Irish English/EDACC-C50-255.wav.txt
Saved audio: organized_data/train/Romanian/EDACC-C30-100.wav
Saved text: organized_data/train/Romanian/E

In [4]:
# Load multiple Whisper models and processors
model_names = ["openai/whisper-tiny", "openai/whisper-base", "openai/whisper-small", "openai/whisper-medium", "openai/whisper-large"]
models = {}
processors = {}
for model_name in model_names:
    processors[model_name] = WhisperProcessor.from_pretrained(model_name)
    models[model_name] = WhisperForConditionalGeneration.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')

# Function to downsample audio samples to 16 kHz
def downsample_audio(audio_array, source_sr, target_sr=16000):
    waveform = torch.tensor(audio_array, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
    resampled_waveform = torchaudio.transforms.Resample(source_sr, target_sr)(waveform)
    resampled_audio = resampled_waveform.squeeze(0).numpy()  # Remove channel dimension
    return resampled_audio

# Function to normalize text
def normalize_text(text):
    text = text.lower()
    text = text.strip()
    text = text.replace("\n", " ")
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove non-alphanumeric characters
    return text

# Function to evaluate WER for each language folder
def evaluate_language_wer(folder_path, processor, model, batch_size=16):
    wer_metric = load_metric("wer", trust_remote_code=True)
    audio_files = [f for f in os.listdir(folder_path) if f.endswith(".wav")]
    predictions = []
    references = []

    for i in range(0, len(audio_files), batch_size):
        batch_files = audio_files[i:i+batch_size]
        input_features = []
        for file in batch_files:
            audio_path = os.path.join(folder_path, file)
            text_path = audio_path + ".txt"
            with open(text_path, "r") as f:
                references.append(normalize_text(f.read().strip()))
            audio, sampling_rate = torchaudio.load(audio_path)
            audio = audio.squeeze().numpy()
            if sampling_rate != 16000:
                audio = downsample_audio(audio, sampling_rate)
            features = processor.feature_extractor(audio, sampling_rate=16000, return_tensors="pt").input_features
            input_features.append(features)

        input_features = torch.cat(input_features).to(model.device)
        with torch.no_grad():
            generated_ids = model.generate(input_features)
        transcriptions = processor.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        predictions.extend([normalize_text(transcription) for transcription in transcriptions])

    wer = wer_metric.compute(predictions=predictions, references=references)
    return wer

# Directory where test data is organized
base_dir = "organized_data"

# Initialize a DataFrame to store WER results
wer_results = pd.DataFrame(columns=["Language"] + model_names)

# Evaluate and print WER for each language and model
for lang in languages:
    folder_path = os.path.join(base_dir, 'test', lang)
    row = {"Language": lang}
    for model_name in model_names:
        processor = processors[model_name]
        model = models[model_name]
        wer = evaluate_language_wer(folder_path, processor, model)
        row[model_name] = wer
        print(f"Language: {lang}, Model: {model_name}, WER: {wer:.4f}")
    wer_results = pd.concat([wer_results, pd.DataFrame([row])], ignore_index=True)

# Display WER results as a table
print("\nWER Results:")
print(wer_results.to_string(index=False))

# Save the WER results to a CSV file
wer_results.to_csv("wer_results.csv", index=False)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

<ipython-input-4-ca8bdf0fbe1e>:27: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer", trust_remote_code=True)


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Language: Scottish English, Model: openai/whisper-tiny, WER: 0.4339
Language: Scottish English, Model: openai/whisper-base, WER: 0.3867
Language: Scottish English, Model: openai/whisper-small, WER: 0.3456
Language: Scottish English, Model: openai/whisper-medium, WER: 0.3121
Language: Scottish English, Model: openai/whisper-large, WER: 0.3054
Language: Sinhalese, Model: openai/whisper-tiny, WER: 0.6822
Language: Sinhalese, Model: openai/whisper-base, WER: 0.4551
Language: Sinhalese, Model: openai/whisper-small, WER: 0.3977
Language: Sinhalese, Model: openai/whisper-medium, WER: 0.6150
Language: Sinhalese, Model: openai/whisper-large, WER: 0.3793
Language: Lithuanian, Model: openai/whisper-tiny, WER: 0.3663
Language: Lithuanian, Model: openai/whisper-base, WER: 0.3770
Language: Lithuanian, Model: openai/whisper-small, WER: 0.3639
Language: Lithuanian, Model: openai/whisper-medium, WER: 0.2698
Language: Lithuanian, Model: openai/whisper-large, WER: 0.3007
Language: Bulgarian, Model: opena